In [1]:
import torch
import numpy as np

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2


Bad key "text.kerning_factor" on line 4 in
/home/deepanshu/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
from torchtext import data , vocab

In [4]:
import torch.nn.functional as F
import torch.nn as nn
import fastai
from fastai.text import*

In [5]:
## returns corresponding lib with default language as en
tokenizer = data.get_tokenizer('spacy') 

In [6]:
## defines datatype that makes tensor and have vocab object
TEXT = data.Field(tokenize = tokenizer , lower = True , eos_token = '_eos_')

In [148]:
# dataset used is gigaword dataset 

In [8]:
trn_data_fields = [("source" , TEXT),
                   ("target" , TEXT)]

trn , vld = data.TabularDataset.splits(path = r'/home/deepanshu/Downloads/' ,train = 'GigaTrainSet.csv' ,
                                       validation = 'GigaValdSet.csv' ,format = 'csv' , skip_header=True , 
                                       fields = trn_data_fields)

In [9]:
print(trn[50].source , trn[50].target)

['tuareg', 'rebels', 'in', 'mali', 'free', 'last', 'hostages'] ['tuareg', 'rebel', 'chief', 'ibrahim', 'ag', 'bahanga', 'has', 'released', 'the', 'last', '#', '#', 'hostages', 'held', 'by', 'his', 'renegade', 'group', 'in', 'the', 'north', 'of', 'mali', ',', 'the', 'government', 'has', 'said', ',', 'in', 'an', 'important', 'step', 'forward', 'towards', 'peace', '.']


In [10]:
# vocab dictionary is made by highest frequency and alphabetic order

In [11]:
pre_trained_vector_type='glove.6B.200d'
TEXT.build_vocab(trn , vectors = pre_trained_vector_type)

In [12]:
TEXT.vocab.freqs.most_common(10)

[('#', 150346),
 ('the', 124206),
 ('.', 95927),
 ('in', 81764),
 (',', 81572),
 ('to', 80665),
 ('of', 75256),
 ('a', 70129),
 ('on', 49036),
 ('-', 40212)]

In [13]:
#iterator that loads batches of data from a Dataset
batch_size =32
train_iter  = data.BucketIterator( trn  , batch_size = batch_size, sort_key = lambda x : len(x.target) ,
                                    shuffle = True , sort_within_batch=False , repeat = False)
valid_iter = data.BucketIterator( vld  , batch_size = batch_size, sort_key = lambda x : len(x.target) ,
                                    shuffle = True , sort_within_batch=False , repeat = False)
                                                     

In [14]:
next(train_iter.__iter__())


[torchtext.data.batch.Batch of size 32]
	[.source]:[torch.LongTensor of size 18x32]
	[.target]:[torch.LongTensor of size 54x32]

In [15]:
next(valid_iter.__iter__())


[torchtext.data.batch.Batch of size 32]
	[.source]:[torch.LongTensor of size 21x32]
	[.target]:[torch.LongTensor of size 50x32]

In [16]:
class BatchTuple():
    def __init__(self , dataset , x_var , y_var):
        self.dataset , self.x_var , self.y_var = dataset , x_var , y_var
        
    def __iter__(self):
        for batch in self.dataset:
            x = getattr(batch , self.x_var)
            y = getattr(batch , self.y_var)
            yield(x ,y)
            
    def __len__(self):
        return len(self.dataset)
    

In [17]:
train_iter_tuple = BatchTuple(train_iter , "source" ,"target")
valid_iter_tuple = BatchTuple(valid_iter , "source" , "target")

In [18]:
next(iter(train_iter_tuple))

(tensor([[ 1105,   172,   639,  1850,  3790,  2262,    64,  4807,  1715,    83,
            305,   329,  1223,  2014,   570,   844,  5783,  4185,  3174,  5027,
             50,  3374,  3830,    64,    69,  2940,   865,   129,   146,    82,
            134,  3108],
         [   81,     6,     9,     9,     8,   457,   216,    39,  5398, 16870,
              9,   127,   591,  1419,  1215,   693,   700,  2548,  3093, 14908,
            196,   675,  5057,   216,  2123,    17,     8,   156,   162,   617,
            878,   246],
         [    6,   455,   133,  3162,  6419,     6,  1982,   131,   176,  2338,
           1085,  2303,   142,  1838,   328,  2443,    16,     6,    79,    51,
            161,  2005,    56,  1982,  3093,    50,   946,  2507,  5591,  7906,
              8,  6390],
         [  601,    40,    13,    44,   133,   339,   159,   322,   484,   223,
            166,     3,   301,   175,    16,    16,  4596,     8,    11,   261,
             64,    11,  2169,   159,   148, 

In [19]:
SampleDataPath = r'/home/deepanshu/Downloads/'


In [20]:
len(train_iter_tuple) , len(valid_iter_tuple) , len(TEXT.vocab)

(2500, 32, 41230)

In [21]:
t ,z = next(train_iter_tuple.__iter__())
t.size() , z.size()

(torch.Size([21, 32]), torch.Size([53, 32]))

In [22]:
sample_source = t.transpose(1,0)[0].data.cpu().numpy()
sample_target = z.transpose(1,0)[0].data.cpu().numpy()

print("Sample Source \n%s \n\nCorresponding Tensor \n%s \n\n" %(' '.join([TEXT.vocab.itos[o] for o in sample_source])
                                                               , sample_source))
print("Sample Target \n%s \n\nCorresponding Tensor \n%s \n\n " %(' '.join([TEXT.vocab.itos[o] for o in sample_target])
                                                               , sample_target))

Sample Source 
economic outlook pushes australian shares # . # percent higher _eos_ <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

Corresponding Tensor 
[ 192 1320 4540  151   77    3    5    3   34  110    2    1    1    1    1    1    1    1    1    1    1] 


Sample Target 
australian shares prices rose by almost half a percent wednesday , buoyed by a healthy outlook for the domestic economy and rises in overseas markets , brokers said . _eos_ <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

Corresponding Tensor 
[151  77  58 360 ...   1   1   1   1] 

 


In [149]:


#### Model

class Seq2Seq(nn.Module):
    def __init__(self , input_size , embz_size , hidden_size , output_size , batch_size , max_tgt_len ,
                 pre_trained_vector ,pre_trained_vector_type ,padding_idx , force_prob = 0.7,
                 encoder_drop =(0,0,0,0) , decoder_drop =(0,0,0,0),num_layers =2 ,
                 teacher_forcing = True , bias = False):
        super().__init__()
        self.output_size , self.embz_size , self.hidden_size = output_size ,embz_size ,hidden_size
        
        self.num_layers = num_layers 
        self.max_tgt_len , self.input_size , self.pretrained_vectors = max_tgt_len ,input_size ,pre_trained_vector
        
        self.teacher_forcing , self.pre_trained_vector_type = teacher_forcing , pre_trained_vector_type
        
        self.encoder_drop , self.decoder_drop , self.padding_idx = encoder_drop ,decoder_drop, padding_idx
        
        if self.teacher_forcing: 
            self.force_prob = force_prob
        
        self.bidirectional = True
        ## since bidirectional
        self.num_directions = 2
        self.beam_width = 5
      
            
        #encoder_
        self.encoder_dropout = nn.Dropout(self.encoder_drop[0])
        self.encoder_embedding_layer = nn.Embedding(self.input_size ,self.embz_size , padding_idx=self.padding_idx)
        if self.pretrained_vectors :
            self.encoder_embedding_layer.weight.data.copy_(self.pretrained_vectors.weight.data)
            
        self.encoder_LSTM = nn.LSTM(input_size = self.embz_size, hidden_size = self.hidden_size,
                                     num_layers = self.num_layers ,dropout = self.encoder_drop[1],
                                    bidirectional = True
                                    )       
        
        self.encoder_vector_layer = nn.Linear(self.hidden_size*self.num_directions ,self.embz_size , bias=bias)
        
        ##decoder
        self.decoder_dropout = nn.Dropout(self.decoder_drop[0])
        self.decoder_embedding_layer = nn.Embedding(self.input_size , self.embz_size , padding_idx=self.padding_idx)
        if self.pretrained_vectors :
            self.decoder_embedding_layer.weight.data.copy_(self.pretrained_vectors.weight.data)
        
        self.decoder_LSTM = nn.LSTM(input_size = self.embz_size, hidden_size = self.hidden_size*self.num_directions,
                                     num_layers = 1 ,dropout = self.decoder_drop[1],
                                    bidirectional = False)
        self.decoder_output_layer = nn.Linear(self.hidden_size*self.num_directions ,self.embz_size , bias=bias)
        self.output_layer = nn.Linear(self.embz_size , self.output_size , bias =bias)
        
        ##set attention
        self.encoder_output_layer =  nn.Linear(self.hidden_size*self.num_directions ,self.embz_size , bias=bias)
        self.attention_vector_layer = nn.Linear(self.embz_size*2 , self.embz_size , bias = bias)
        self.decoder_hidden_layer = nn.Linear(self.hidden_size*self.num_directions ,self.embz_size , bias=bias)
        self.attn_score = nn.Linear(self.embz_size , 1 , bias=bias)
        
        
    def init_hidden(self , batch_size):
        return (torch.zeros(self.num_layers*self.num_directions , batch_size , self.hidden_size) ,
                    torch.zeros(self.num_layers*self.num_directions , batch_size , self.hidden_size) )
       
    
    def cat_directions(self , hidden):
        def cat_(h):
            #changes 2*l into l(layers) and concat at hidden_size (3rd dim.) so that we can extract last layer
            return torch.cat([h[0:h.size(0):2] , h[1:h.size(0):2]] ,2 )
            
        hidden = tuple([cat_(h) for h in hidden])
        return hidden
    
    
    def attention(self ,encoder_output,decoder_hidden,decoder_input):
        encoder_out = self.encoder_output_layer(encoder_output)  #(seqlen , batch , embz)
        encoder_out = encoder_out.transpose(1,0)                 #(batch , seqlen , embz)
        decoder_hidden = decoder_hidden.transpose(1,0)          # (batch , 1 ,embz)
        att_score = torch.tanh(encoder_out + decoder_hidden)
        att_score = self.attn_score(att_score)                   #(batch , seqlen ,1)
        att_weight = F.softmax(att_score , dim=1)
        context_vector = torch.bmm(att_weight.transpose(-1,1) ,encoder_out ).squeeze(1)  #(batch  ,embz)
        att_vector = torch.cat((context_vector ,decoder_input ) , dim=1)                 #(batch , 2*embz)
        att_vector = self.attention_vector_layer(att_vector)                             #(batch , embz)
        return att_weight.squeeze(-1) , att_vector
            
    
    # Make one step of decoder lstm 
        
    def dec_lstm_step(self , dec_input , encoder_output , decoder_hidden ) :
        dec_input = self.decoder_dropout(self.decoder_embedding_layer(dec_input))
        prev_hidden = self.decoder_hidden_layer(decoder_hidden[0][-1]).unsqueeze(0)
        #(1,batch,embz)       (batch,embz)      last layer of (l,batch,2*hidden_size)
        att , decoder_input = self.attention(encoder_output , prev_hidden , dec_input)
        #(batch , seqlen)  , (batch ,  embz)
        decoder_hidden = (decoder_hidden[0][-1].unsqueeze(0) ,decoder_hidden[1][-1].unsqueeze(0) )  
        #give the top layer states to decoder_LSTM
        decoder_output , decoder_hidden = self.decoder_LSTM(decoder_input.unsqueeze(0) , decoder_hidden)
        #(1,batch,2*hidden) ((1,batch,2*hidden_size),(1,batch,2*hidden_size))
        
        decoder_output = self.decoder_output_layer(decoder_output.squeeze(0))
        #(batch , embz)
        output = self.output_layer(decoder_output)   #(batch , output_size) 
        output = F.softmax(output , dim=1)
        return att , output ,decoder_hidden
    
    
    # for training 
        
    def decoder_forward(self , batch_size, encoder_output , decoder_hidden , force_prob =0.7, y=None ):
        self.force_prob = force_prob
        dec_input = torch.zeros(batch_size).long()
        output_seq_stack , att_stack = [] , []
        for i in range(self.max_tgt_len):
            att , output , decoder_hidden = self.dec_lstm_step( dec_input , encoder_output , decoder_hidden )
            
            att_stack.append(att)
            
            output_seq_stack.append(output)
            decoder_input = output.data.max(1)[1]     #(batch)   
            
            if self.teacher_forcing:
                samp_prob = round(random.random() ,1)
                if (y is not None) and (samp_prob<self.force_prob):     #condition y not None is always true here
                    if i >= len(y):                                                    
                        break
                    decoder_input = y[i]
                    
            if (decoder_input==1).all(): 
                break 
            
        return torch.stack(output_seq_stack) , torch.stack(att_stack)  
                   #(max_tgt_len,batch,output_size)  , (max_tgt_len,batch,seqlen)  
                
    
    
    # for inference 
    
    def beam_search_inference(self, batch_size , beam_width, encoder_output , decoder_hidden):
        dec_input = torch.zeros(batch_size).long()                           
        final_score , sent , indices_list = [] ,[] ,[]
        minus_inf_output = torch.ones(batch_size , output_size)*(-float("Inf"))
        att , output,decoder_hidden = self.dec_lstm_step(dec_input, encoder_output , decoder_hidden)  
        #output.size()=(batch,output_size) decoder_hidden.size()=((1,batch,2*hidden_size),(1,batch,2*hidden_size))
        h,c = decoder_hidden
        h_list = [h.squeeze(0) for k in range(beam_width)]              #each element of list is h squeezed (batch,2*hiiden_size)
        c_list = [c.squeeze(0) for k in range(beam_width)]
        
        score = torch.log(output.topk(beam_width , dim =1)[0])          #take log of prob of top words
        
        for i in range(self.max_tgt_len -1):                            # -1 as one step has already been taken
            indices = output.topk(beam_width , dim =1)[1]               #get indices with top prob.(batch,beam_width)
            indices_source = torch.floor((indices)*(1/output_size))     # get the source of top scores
            indices_char = (indices)%(output_size)                      # get chars of top scorers
            char_source = zip(indices_source ,indices_char)             #zip the source and chars so that later 
            indices_list.append(char_source)                            #sentence can be backtracked 
            
            beam_score , decoder_hidden_list= [] ,[]                                             
            for j in range(beam_width):                              
                dec_input = indices_char[:,j]                           #give each top indices character as input one by one
                if(dec_input ==1).all():                                #if input is <pad>
                    score[:,j] = score[:,j]*(1/(i+1))                   #normalize the score by length(+1 to include length till <pad>)
                                                                        #as the score contains <pad> prob also
                    final_score.append(score[:,j])                      #make list of all scores so that can be compared later
                    sent.append((i,j))                                  #take note of (i,j) for sentence retrivel
                    beam_score.append(minus_inf_output)                 #append a dummy output having -inf log(prob) and  
                                                                        #decoder hidden_state with None so that
                    decoder_hidden_list.append((None,None))             #source can be assigned properly to other input  
                    
                    continue
                
                ind_src = indices_source[:,j]                           #get the source of the dec_input for decoder_hidden
                temp_h = [h_list[int(ind_src[bt])][bt]  for bt in range(batch_size)]
                temp_c = [c_list[int(ind_src[bt])][bt]  for bt in range(batch_size)]
                #make a list of tensors of the corresponding decoder_hidden states for each element of batch
                
                h = torch.stack(temp_h)                                #make it tensor (1,batch,2*hidden_state)
                h = h.unsqueeze(0)
                
                c = torch.stack(temp_c)                                #so that it can be passed to lstm
                c = c.unsqueeze(0)
                
                decoder_hidden = (h,c)
                att , output ,decoder_hidden_new = self.dec_lstm_step(dec_input , encoder_output , decoder_hidden)
                
                decoder_hidden_list.append(decoder_hidden_new)         #list of tuples 
                output = torch.log(output)                             #log of the new outputs given j-th indices as input 
                sc_output = output + score[:,j].unsqueeze(1)           #add the previous log prob to it
                beam_score.append(sc_output)                           #make a list of all possible outputs so that top
                                                                       #prob can be extracted among all inputs
                
            output = torch.cat(beam_score , dim =1)                    #make a tensor from the list
            if (output==-float("Inf")).all():                          #if every input is <pad> then break because no new
                break                                                  #output will be genrated and score will be divided
                                                                       # by i+1 again and again
            score = output.topk(beam_width , dim =1)[0]                #find the top scores among all possible output
            h_list = [decoder_hidden_list[k][0].squeeze(0) if decoder_hidden_list[k][0] is not None  else None   for k in range(beam_width)]
            c_list = [decoder_hidden_list[k][1].squeeze(0) if decoder_hidden_list[k][1] is not None  else None   for k in range(beam_width)]
            #update the list of h,c so that for the next input appropriate decoder state can be given as input
            #using the sources of the input indices
            
        #if no <pad> comes in between and to consider incomplete sentences that reached max length
        for j in range(beam_width):
            score[:,j] = score[:,j]*(1/(i+2))                         #consider all top scores , normalize by length
            final_score.append(score[:,j])
            sent.append((i+1,j))
            
        indices = output.topk(beam_width , dim =1)[1]
        indices_source = torch.floor((indices)*(1/output_size))
        indices_char = (indices)%(output_size)
        char_source = zip(indices_source ,indices_char)
        indices_list.append(char_source)                          #appended at i+1 position
            
                
        
        
        return indices_list , final_score , sent
                
    
    #make sentence for a batch element            
    
    def sentence_maker(self,idx , bt , sent , indices_list ):
        i, j = sent[idx]                    # i is used for length and j to get the char from possible top indices
        sentence = []
        for x in range(i , -1 ,-1):
            indices = indices_list[x]       #get the char and source at x position in list
            src , char = zip(*indices)      #unzip to get the char and its source
            sent_idx = char[bt][int(j)]          
            sent_src = src[bt][int(j)]           #get the char and source for this batch element
            sentence.append(sent_idx)       
            j = sent_src                    #now get the char at this char's source position
            
            indices_list[x] = zip(src,char) #zip it again to be used by other batch elements
            
        sentence = torch.FloatTensor(sentence)
        sentence = sentence.flip(0)             #flip the sentence as list was made by backtracking
        
        return sentence                     #return sentence tensor
    
    
    #main function 
     
    def sentence_inference(self , batch_size,beam_width ,encoder_output, decoder_hidden):
        beam_width = beam_width
        indices_list, final_score, sent = self.beam_search_inference(batch_size,beam_width ,encoder_output ,decoder_hidden)
        batch_sentence = [] 
        for bt in range(batch_size):
            scr = [final_score[i][bt] for i in range(len(final_score))]  #for each element from batch make list of  
            idx = scr.index(max(scr))                                    #final_scores of different sentence and get
                                                                         #the max score
            sentence = self.sentence_maker(idx , bt , sent , indices_list)    #get the sentence for current element of batch
            batch_sentence.append(sentence)                              #make list of all sentences
            
        return batch_sentence                                            #return list of sentences tensor
            
            
    
        
        
    def forward(self , seq , y=None , beam_width = 5 ,force_prob = 0.7):
        self.force_prob = force_prob
        self.beam_width = beam_width
        batch_size = seq[0].size(0)
        encoder_hidden = self.init_hidden(batch_size)
        encoder_input = self.encoder_dropout(self.encoder_embedding_layer(seq))   #(seqlen , batch , embz)
        encoder_output , encoder_hidden = self.encoder_LSTM(encoder_input ,encoder_hidden) 
        # (seqlen,batch,2*hiddensize) , (2*l , batch , hidden_size)
            
        encoder_hidden = self.cat_directions(encoder_hidden)    #(l , batch , 2*hidden_size)
        if y is not None:
            output = self.decoder_forward(batch_size ,encoder_output ,encoder_hidden ,self.force_prob, y=y )
            #training phase ,here output is tuple 
            
        else:
            output = self.sentence_inference(batch_size,self.beam_width , encoder_output,encoder_hidden)
            # inference phase , here output is list of all sentences
            
        return output
            
        
        
        
        

In [24]:
def save_pickle(path, filename , file):
    with open(f'{path}/{filename}' , 'wb') as f:
        pickle.dump(file , f)
        

def norm_pretrained_embeddings(vecs , itos ,em_sz , padding_idx,vec_mean,vec_std):
    #to make length of each vector equal and normalize them
    emb = nn.Embedding(len(itos) , em_sz , padding_idx =padding_idx)
    wgt = emb.weight.data
    for i,w in enumerate(itos):
        
        wgt[i] = ((vecs[i] - vec_mean)/vec_std)      # vocab.vectors are used from index of char
                    
    emb.weight.requires_grad = False
    return emb
    
def embedding_param(path, data_field, pre_trained_vector_type, embz_size=128, save_vocab=False, itos='itos',
                    stoi='stoi'):
    # to copy glove vectors from TEXT to pre_trained_vector(matrix and obtained after normalizing) which will
    # be passed to model so that encoder_embeding can copy data from it
    pre_trained = None
    padding_idx = data_field.vocab.stoi['<pad>']
    index_to_string , string_to_index = data_field.vocab.itos , data_field.vocab.stoi
    if save_vocab:
        vocab_path = os.path.join(path , "vocab")
        os.makedirs(vocab_path, exist_ok=True)
        save_pickle(vocab_path, f'{itos}.pk', index_to_string)
        save_pickle(vocab_path ,f'{stoi}.pk' , string_to_index)
    if pre_trained_vector_type:
        vec_mean , vec_std = data_field.vocab.vectors.numpy().mean() , data_field.vocab.vectors.numpy().std()
    print('pre_trained_vector_type mean = %s , pre_trained_vector_type std = %s' %(vec_mean ,vec_std)) 
    vector_wt_matrix = data_field.vocab.vectors
    embz_size = vector_wt_matrix.size(1)
    print('vector_wt_matrix size : \n' ,(vector_wt_matrix.size()))
    
    pre_trained = norm_pretrained_embeddings(vector_wt_matrix ,index_to_string ,embz_size ,padding_idx,vec_mean,vec_std)
    print('Normalizing... \npre_trained_vector_mean = %s, pre_trained_vector_std=%s '
          %(pre_trained.weight.data.numpy().mean() , pre_trained.weight.data.numpy().std()))
    return pre_trained , embz_size ,padding_idx 
        
        

In [25]:
pre_trained_vector , embz_size , padding_idx = embedding_param(SampleDataPath, TEXT ,pre_trained_vector_type ,
                                                              save_vocab=True)

/home/deepanshu/anaconda3/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


pre_trained_vector_type mean = 0.0026343681 , pre_trained_vector_type std = 0.43798193
vector_wt_matrix size : 
 torch.Size([41230, 200])
Normalizing... 
pre_trained_vector_mean = 3.321553e-09, pre_trained_vector_std=0.99999994 


In [119]:
its = [next(train_iter_tuple.__iter__())[0]  for i in range(20)]
max_tgt_len = int(np.percentile([its[o].size(0) for o in range(len(its))] , 99))
max_tgt_len

18

In [141]:
input_size = len(TEXT.vocab)
hidden_size = 200
output_size = len(TEXT.vocab)
max_tgt_len = max_tgt_len
beam_width = 5

In [28]:
def seq2seqloss(output , target):
    sl,bs = target.size()
    sl_in,bs_in,nc = output.size()           #nc is no of classes or words = output_size =len(TEXT.vocab)
    if sl>sl_in: output = F.pad(output, (0,0,0,0,0,sl-sl_in))
    output = output[:sl]
    return F.cross_entropy(output.view(-1,nc), target.view(-1))

In [150]:
 model = Seq2Seq(input_size , embz_size , hidden_size, output_size ,batch_size ,max_tgt_len,pre_trained_vector,
                pre_trained_vector_type, padding_idx )

In [30]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [33]:
epochs = 7

In [34]:
for epoch in range(epochs):
    running_loss =0
    force_prob = (10 - epoch)*0.1 if epoch<10 else 0
   
    for i , data in enumerate(train_iter_tuple):
        summary , story = data
        optimizer.zero_grad()
        prob = model(story , summary ,beam_width, force_prob)
        loss = seq2seqloss(prob[0] , summary)
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
        if i%10 == 9:  #print every 10th minibatch
            print('[%d , %5d] loss = %.3f' %(epoch+1 , i+1 , running_loss/10))
            running_loss =0
            
print('Finished Training')
            
    
    
    
    
    
    
    

[1 ,    10] loss = 4.281
[1 ,    20] loss = 3.962
[1 ,    30] loss = 4.405
[1 ,    40] loss = 3.817
[1 ,    50] loss = 3.983
[1 ,    60] loss = 4.462
[1 ,    70] loss = 4.131
[1 ,    80] loss = 4.664
[1 ,    90] loss = 4.628
[1 ,   100] loss = 4.521
[1 ,   110] loss = 4.214
[1 ,   120] loss = 4.328
[1 ,   130] loss = 4.530
[1 ,   140] loss = 4.392
[1 ,   150] loss = 4.188
[1 ,   160] loss = 4.166
[1 ,   170] loss = 4.413
[1 ,   180] loss = 4.330
[1 ,   190] loss = 4.410
[1 ,   200] loss = 4.515
[1 ,   210] loss = 4.319
[1 ,   220] loss = 4.306
[1 ,   230] loss = 4.599
[1 ,   240] loss = 4.130
[1 ,   250] loss = 4.279
[1 ,   260] loss = 4.680
[1 ,   270] loss = 4.674
[1 ,   280] loss = 4.569
[1 ,   290] loss = 4.093
[1 ,   300] loss = 4.179
[1 ,   310] loss = 5.160
[1 ,   320] loss = 4.474
[1 ,   330] loss = 3.852
[1 ,   340] loss = 3.955
[1 ,   350] loss = 4.418
[1 ,   360] loss = 4.518
[1 ,   370] loss = 4.325
[1 ,   380] loss = 4.027
[1 ,   390] loss = 4.838
[1 ,   400] loss = 4.452


[2 ,   790] loss = 4.047
[2 ,   800] loss = 3.680
[2 ,   810] loss = 3.684
[2 ,   820] loss = 3.827
[2 ,   830] loss = 3.922
[2 ,   840] loss = 4.875
[2 ,   850] loss = 4.366
[2 ,   860] loss = 3.949
[2 ,   870] loss = 4.362
[2 ,   880] loss = 4.396
[2 ,   890] loss = 3.759
[2 ,   900] loss = 3.848
[2 ,   910] loss = 4.662
[2 ,   920] loss = 4.113
[2 ,   930] loss = 3.666
[2 ,   940] loss = 4.683
[2 ,   950] loss = 3.938
[2 ,   960] loss = 3.901
[2 ,   970] loss = 4.147
[2 ,   980] loss = 3.812
[2 ,   990] loss = 4.109
[2 ,  1000] loss = 4.081
[2 ,  1010] loss = 4.957
[2 ,  1020] loss = 4.193
[2 ,  1030] loss = 3.954
[2 ,  1040] loss = 4.400
[2 ,  1050] loss = 4.019
[2 ,  1060] loss = 4.085
[2 ,  1070] loss = 4.417
[2 ,  1080] loss = 4.643
[2 ,  1090] loss = 3.686
[2 ,  1100] loss = 4.119
[2 ,  1110] loss = 3.505
[2 ,  1120] loss = 3.443
[2 ,  1130] loss = 4.293
[2 ,  1140] loss = 4.882
[2 ,  1150] loss = 4.155
[2 ,  1160] loss = 4.001
[2 ,  1170] loss = 3.821
[2 ,  1180] loss = 4.807


[3 ,  1570] loss = 3.550
[3 ,  1580] loss = 3.567
[3 ,  1590] loss = 4.345
[3 ,  1600] loss = 4.325
[3 ,  1610] loss = 3.880
[3 ,  1620] loss = 4.005
[3 ,  1630] loss = 3.329
[3 ,  1640] loss = 3.483
[3 ,  1650] loss = 4.078
[3 ,  1660] loss = 3.792
[3 ,  1670] loss = 3.837
[3 ,  1680] loss = 3.949
[3 ,  1690] loss = 4.085
[3 ,  1700] loss = 3.964
[3 ,  1710] loss = 4.950
[3 ,  1720] loss = 3.425
[3 ,  1730] loss = 3.680
[3 ,  1740] loss = 3.992
[3 ,  1750] loss = 3.687
[3 ,  1760] loss = 4.305
[3 ,  1770] loss = 3.820
[3 ,  1780] loss = 3.955
[3 ,  1790] loss = 3.504
[3 ,  1800] loss = 3.255
[3 ,  1810] loss = 3.303
[3 ,  1820] loss = 4.212
[3 ,  1830] loss = 4.115
[3 ,  1840] loss = 4.470
[3 ,  1850] loss = 3.629
[3 ,  1860] loss = 3.620
[3 ,  1870] loss = 4.508
[3 ,  1880] loss = 3.740
[3 ,  1890] loss = 4.784
[3 ,  1900] loss = 4.049
[3 ,  1910] loss = 3.361
[3 ,  1920] loss = 3.810
[3 ,  1930] loss = 4.435
[3 ,  1940] loss = 3.493
[3 ,  1950] loss = 4.179
[3 ,  1960] loss = 3.753


[4 ,  2350] loss = 3.330
[4 ,  2360] loss = 3.592
[4 ,  2370] loss = 4.151
[4 ,  2380] loss = 3.927
[4 ,  2390] loss = 3.319
[4 ,  2400] loss = 3.478
[4 ,  2410] loss = 3.466
[4 ,  2420] loss = 3.641
[4 ,  2430] loss = 3.919
[4 ,  2440] loss = 3.351
[4 ,  2450] loss = 3.447
[4 ,  2460] loss = 4.045
[4 ,  2470] loss = 3.841
[4 ,  2480] loss = 3.276
[4 ,  2490] loss = 3.762
[4 ,  2500] loss = 3.392
[5 ,    10] loss = 3.702
[5 ,    20] loss = 2.754
[5 ,    30] loss = 2.951
[5 ,    40] loss = 2.806
[5 ,    50] loss = 2.818
[5 ,    60] loss = 2.718
[5 ,    70] loss = 2.568
[5 ,    80] loss = 3.355
[5 ,    90] loss = 2.490
[5 ,   100] loss = 3.158
[5 ,   110] loss = 3.430
[5 ,   120] loss = 3.243
[5 ,   130] loss = 2.989
[5 ,   140] loss = 3.097
[5 ,   150] loss = 3.140
[5 ,   160] loss = 3.000
[5 ,   170] loss = 3.005
[5 ,   180] loss = 3.120
[5 ,   190] loss = 2.868
[5 ,   200] loss = 2.744
[5 ,   210] loss = 2.813
[5 ,   220] loss = 2.975
[5 ,   230] loss = 2.880
[5 ,   240] loss = 3.439


[6 ,   630] loss = 2.823
[6 ,   640] loss = 2.768
[6 ,   650] loss = 2.581
[6 ,   660] loss = 2.800
[6 ,   670] loss = 2.820
[6 ,   680] loss = 2.619
[6 ,   690] loss = 2.626
[6 ,   700] loss = 2.830
[6 ,   710] loss = 2.809
[6 ,   720] loss = 2.788
[6 ,   730] loss = 2.695
[6 ,   740] loss = 2.893
[6 ,   750] loss = 2.802
[6 ,   760] loss = 3.193
[6 ,   770] loss = 2.977
[6 ,   780] loss = 2.821
[6 ,   790] loss = 2.501
[6 ,   800] loss = 2.915
[6 ,   810] loss = 3.202
[6 ,   820] loss = 3.051
[6 ,   830] loss = 3.098
[6 ,   840] loss = 2.941
[6 ,   850] loss = 2.687
[6 ,   860] loss = 3.208
[6 ,   870] loss = 2.912
[6 ,   880] loss = 3.111
[6 ,   890] loss = 2.932
[6 ,   900] loss = 2.953
[6 ,   910] loss = 2.908
[6 ,   920] loss = 2.888
[6 ,   930] loss = 3.245
[6 ,   940] loss = 2.584
[6 ,   950] loss = 2.581
[6 ,   960] loss = 2.973
[6 ,   970] loss = 2.717
[6 ,   980] loss = 3.074
[6 ,   990] loss = 2.692
[6 ,  1000] loss = 2.735
[6 ,  1010] loss = 2.786
[6 ,  1020] loss = 2.763


[7 ,  1410] loss = 3.077
[7 ,  1420] loss = 2.420
[7 ,  1430] loss = 2.637
[7 ,  1440] loss = 2.582
[7 ,  1450] loss = 2.544
[7 ,  1460] loss = 2.414
[7 ,  1470] loss = 3.045
[7 ,  1480] loss = 2.886
[7 ,  1490] loss = 2.730
[7 ,  1500] loss = 2.771
[7 ,  1510] loss = 2.259
[7 ,  1520] loss = 2.634
[7 ,  1530] loss = 2.587
[7 ,  1540] loss = 2.724
[7 ,  1550] loss = 2.580
[7 ,  1560] loss = 2.851
[7 ,  1570] loss = 2.730
[7 ,  1580] loss = 2.581
[7 ,  1590] loss = 2.490
[7 ,  1600] loss = 2.590
[7 ,  1610] loss = 2.451
[7 ,  1620] loss = 2.258
[7 ,  1630] loss = 2.407
[7 ,  1640] loss = 2.639
[7 ,  1650] loss = 2.530
[7 ,  1660] loss = 2.437
[7 ,  1670] loss = 2.528
[7 ,  1680] loss = 2.657
[7 ,  1690] loss = 2.747
[7 ,  1700] loss = 2.973
[7 ,  1710] loss = 2.404
[7 ,  1720] loss = 2.517
[7 ,  1730] loss = 2.632
[7 ,  1740] loss = 2.623
[7 ,  1750] loss = 2.796
[7 ,  1760] loss = 2.390
[7 ,  1770] loss = 2.767
[7 ,  1780] loss = 2.707
[7 ,  1790] loss = 2.303
[7 ,  1800] loss = 2.597


In [35]:
index_to_string = TEXT.vocab.itos
string_to_index = TEXT.vocab.stoi

In [36]:
torch.save(model.state_dict() , 'SavedTxtSumModel.pth' )

In [151]:
#to get back model parameters
model.load_state_dict(torch.load('SavedTxtSumModel.pth'))
model.eval()

Seq2Seq(
  (pretrained_vectors): Embedding(41230, 200, padding_idx=1)
  (encoder_dropout): Dropout(p=0, inplace=False)
  (encoder_embedding_layer): Embedding(41230, 200, padding_idx=1)
  (encoder_LSTM): LSTM(200, 200, num_layers=2, bidirectional=True)
  (encoder_vector_layer): Linear(in_features=400, out_features=200, bias=False)
  (decoder_dropout): Dropout(p=0, inplace=False)
  (decoder_embedding_layer): Embedding(41230, 200, padding_idx=1)
  (decoder_LSTM): LSTM(200, 400)
  (decoder_output_layer): Linear(in_features=400, out_features=200, bias=False)
  (output_layer): Linear(in_features=200, out_features=41230, bias=False)
  (encoder_output_layer): Linear(in_features=400, out_features=200, bias=False)
  (attention_vector_layer): Linear(in_features=400, out_features=200, bias=False)
  (decoder_hidden_layer): Linear(in_features=400, out_features=200, bias=False)
  (attn_score): Linear(in_features=200, out_features=1, bias=False)
)

In [38]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=3,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [155]:
#evaluate results in validation dataset
no_of_example = len(vld)
SCORE = {'rouge-2': {'f_score': 0.0, 'p_score': 0.0, 'r_score': 0.0},      #initialize rouge score of validation 
         'rouge-3': {'f_score': 0.0, 'p_score': 0.0, 'r_score': 0.0},      #set with 0
         'rouge-1': {'f_score': 0.0, 'p_score': 0.0, 'r_score': 0.0},
         'rouge-l': {'f_score': 0.0, 'p_score': 0.0, 'r_score': 0.0},
         'rouge-w': {'f_score': 0.0, 'p_score': 0.0, 'r_score': 0.0}}


for i , data  in enumerate(valid_iter_tuple):
    summary , article = data
    out = model(article , None , beam_width)     #out is list 
    for k in range(len(out)):
        
        preds = out[k]
        result = ' '.join([index_to_string[int(o)] for o in preds.data.numpy() if o!=1])
        orig = ' '.join([index_to_string[o] for o in summary[:,k].data.cpu().numpy() if o!=1])
        
        scores = evaluator.get_scores(result, orig)
        SCORE['rouge-2']['f_score'] += scores['rouge-2']['f']/no_of_example         #take average of corresponding 
        SCORE['rouge-2']['p_score'] += scores['rouge-2']['p']/no_of_example         #scores for final validation SCORE
        SCORE['rouge-2']['r_score'] += scores['rouge-2']['r']/no_of_example
        
        SCORE['rouge-3']['f_score'] += scores['rouge-3']['f']/no_of_example
        SCORE['rouge-3']['p_score'] += scores['rouge-3']['p']/no_of_example
        SCORE['rouge-3']['r_score'] += scores['rouge-3']['r']/no_of_example 
        
        SCORE['rouge-1']['f_score'] += scores['rouge-1']['f']/no_of_example
        SCORE['rouge-1']['p_score'] += scores['rouge-1']['p']/no_of_example
        SCORE['rouge-1']['r_score'] += scores['rouge-2']['r']/no_of_example
        
        SCORE['rouge-l']['f_score'] += scores['rouge-l']['f']/no_of_example
        SCORE['rouge-l']['p_score'] += scores['rouge-l']['p']/no_of_example
        SCORE['rouge-l']['r_score'] += scores['rouge-l']['r']/no_of_example
        
        SCORE['rouge-w']['f_score'] += scores['rouge-w']['f']/no_of_example
        SCORE['rouge-w']['p_score'] += scores['rouge-w']['p']/no_of_example
        SCORE['rouge-w']['r_score'] += scores['rouge-w']['r']/no_of_example
        
        
        
print('ROUGE SCORE ON VALIDATION SET')
SCORE


        
        

    
    

ROUGE SCORE ON VALIDATION SET


{'rouge-2': {'f_score': 0.05756711733100212,
  'p_score': 0.06079366189366196,
  'r_score': 0.0571871434121435},
 'rouge-3': {'f_score': 0.01799197322285557,
  'p_score': 0.01909047619047619,
  'r_score': 0.017923917748917745},
 'rouge-1': {'f_score': 0.31273702030367223,
  'p_score': 0.3335660506160498,
  'r_score': 0.0571871434121435},
 'rouge-l': {'f_score': 0.3640601423673266,
  'p_score': 0.38275283636454716,
  'r_score': 0.35888625357357584},
 'rouge-w': {'f_score': 0.21103159077033953,
  'p_score': 0.2873979701562313,
  'r_score': 0.17396290005154413}}

In [253]:
x,y = next(valid_iter_tuple.__iter__())
out = model(y.transpose(1,0)[0].unsqueeze(1) , None , beam_width)     #out is list
preds = out[0]

sentence = ' '.join([index_to_string[o] for o in y[:,0].data.cpu().numpy() if o!=1])
result = ' '.join([index_to_string[int(o)] for o in preds.data.numpy() if o!=1 ])
orig = ' '.join([index_to_string[o] for o in x[:,0].data.cpu().numpy() if o!=1])

print("\nSentence : \n%s" %(sentence))
print("\nResult : \n%s" %(result))
print("\nOriginal : \n%s" %(orig))



Sentence : 
thailand will discuss lifting its ban on us beef , imposed two years ago over mad cow fears , when negotiators from the two nations meet next week for free trade talks , health officials said thursday . _eos_

Result : 
thailand to to lifting beef beef chicken chicken chicken _eos_ _eos_ _eos_

Original : 
thailand may lift ban on us beef _eos_


In [207]:


scores = evaluator.get_scores(result, orig)
print('ROUGE SCORES :') 
scores


ROUGE SCORES :


{'rouge-2': {'f': 0.888888888888889, 'p': 0.8, 'r': 1.0},
 'rouge-1': {'f': 0.9090909090909091, 'p': 0.8333333333333334, 'r': 1.0},
 'rouge-3': {'f': 0.5714285714285715, 'p': 0.5, 'r': 0.6666666666666666},
 'rouge-l': {'f': 0.9241784847394879, 'p': 0.8590444340720371, 'r': 1.0},
 'rouge-w': {'f': 0.7752750336123038,
  'p': 0.8333333333333333,
  'r': 0.7247796636776956}}